# Intro. to Snorkel: Extracting Spouse Relations from the News

## Part III: Training an End Extraction Model

In this final section of the tutorial, we'll use the noisy training labels we generated in the last tutorial part to train our end extraction model.

For this tutorial, we will be training a Bi-LSTM, a state-of-the-art deep neural network implemented in [TensorFlow](https://www.tensorflow.org/).

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()

We repeat our definition of the `Spouse` `Candidate` subclass:

In [2]:
from snorkel.models import candidate_subclass

Spouse = candidate_subclass('Spouse', ['person1', 'person2'])

We reload the probabilistic training labels:

In [3]:
from snorkel.annotations import load_marginals

train_marginals = load_marginals(session, split=0)

We also reload the candidates:

In [4]:
train_cands = session.query(Spouse).filter(Spouse.split == 0).order_by(Spouse.id).all()
dev_cands   = session.query(Spouse).filter(Spouse.split == 1).order_by(Spouse.id).all()
test_cands  = session.query(Spouse).filter(Spouse.split == 2).order_by(Spouse.id).all()

Finally, we load gold labels for evaluation:

In [5]:
from snorkel.annotations import load_gold_labels

L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, load_as_array=True, zero_one=True)
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2, zero_one=True)

Now we can setup our discriminative model. Here we specify the model and learning hyperparameters.

They can also be set automatically using a search based on the dev set with a [GridSearch](https://github.com/HazyResearch/snorkel/blob/master/snorkel/learning/utils.py) object.

In [6]:
from snorkel.learning.disc_models.rnn import reRNN

train_kwargs = {
    'lr':         0.001,
    'dim':        100,
    'n_epochs':   20,
    'dropout':    0.5,
    'print_freq': 1,
    'max_sentence_length': 100
}

lstm = reRNN(seed=1701, n_threads=None)
lstm.train(train_cands, train_marginals, X_dev=dev_cands, Y_dev=L_gold_dev, **train_kwargs)

[reRNN] Training model
[reRNN] n_train=34296  #epochs=20  batch size=256
[reRNN] Epoch 0 (85.85s)	Average loss=0.567191	Dev F1=35.12
[reRNN] Epoch 1 (179.16s)	Average loss=0.541023	Dev F1=39.68
[reRNN] Epoch 2 (283.20s)	Average loss=0.540167	Dev F1=39.66
[reRNN] Epoch 3 (389.42s)	Average loss=0.539933	Dev F1=39.17
[reRNN] Epoch 4 (482.78s)	Average loss=0.539635	Dev F1=41.23
[reRNN] Epoch 5 (583.51s)	Average loss=0.539433	Dev F1=40.93
[reRNN] Epoch 6 (696.77s)	Average loss=0.539339	Dev F1=42.27
[reRNN] Epoch 7 (809.53s)	Average loss=0.539247	Dev F1=42.57
[reRNN] Epoch 8 (927.18s)	Average loss=0.539030	Dev F1=42.33
[reRNN] Epoch 9 (1037.30s)	Average loss=0.538950	Dev F1=45.38
[reRNN] Epoch 10 (1138.58s)	Average loss=0.538891	Dev F1=43.43
[reRNN] Epoch 11 (1242.14s)	Average loss=0.538822	Dev F1=43.71
[reRNN] Epoch 12 (1352.60s)	Average loss=0.538749	Dev F1=44.29
[reRNN] Epoch 13 (1452.46s)	Average loss=0.538705	Dev F1=42.32
[reRNN] Epoch 14 (1553.29s)	Average loss=0.538673	Dev F1=45.02
[r

Now, we get the precision, recall, and F1 score from the discriminative model:

In [7]:
p, r, f1 = lstm.score(test_cands, L_gold_test)
print("Prec: {0:.3f}, Recall: {1:.3f}, F1 Score: {2:.3f}".format(p, r, f1))

Prec: 0.426, Recall: 0.627, F1 Score: 0.507


We can also get the candidates returned in sets (true positives, false positives, true negatives, false negatives) as well as a more detailed score report:

In [8]:
tp, fp, tn, fn = lstm.error_analysis(session, test_cands, L_gold_test)

Scores (Un-adjusted)
Pos. class accuracy: 0.627
Neg. class accuracy: 0.923
Precision            0.426
Recall               0.627
F1                   0.507
----------------------------------------
TP: 281 | FP: 379 | TN: 4567 | FN: 167



Note that if this is the final test set that you will be reporting final numbers on, to avoid biasing results you should not inspect results.  However you can run the model on your _development set_ and, as we did in the previous part with the generative labeling function model, inspect examples to do error analysis.

You can also improve performance substantially by increasing the number of training epochs!

Finally, we can save the predictions of the model on the test set back to the database. (This also works for other candidate sets, such as unlabeled candidates.)

In [ ]:
lstm.save_marginals(session, test_cands)

##### More importantly, you completed the introduction to Snorkel! Give yourself a pat on the back!